# dev - John Ragland

## I'm playing around to create a quick script to DAS to Zarr

- specifically trying to get this experiment into zarr: http://piweb.ooirsn.uw.edu/das/data/Optasense/SouthCable/TransmitFiber/South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T16_09_15-0700/

- chunk size is arbitrarily chosen to be {'time':3000, 'distance':3000}
- quick back of the envelope calculations: (single file experimented with)
    - Dataset should be ~ 4.659 TB stored in hdf5
    - Dataset should be ~ ___ TB stored in zarr
    
    
- I'm implementing this in xarray because that's what I know how to use better. I'm not sure if this is causing a significant increase in time or not.

In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import os
import h5py
from tqdm import tqdm

from dask.distributed import Client

In [2]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 32,Total memory: 62.80 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33497,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 32
Started: Just now,Total memory: 62.80 GiB
Comm: tcp://127.0.0.1:37965,Total threads: 4
Dashboard: http://127.0.0.1:43597/status,Memory: 7.85 GiB
Nanny: tcp://127.0.0.1:44545,


### Single File Test

In [3]:
f = h5py.File('/das_data/South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T231514Z.h5', 'r')

In [4]:
f['Acquisition']['Custom']

<HDF5 group "/Acquisition/Custom" (0 members)>

In [5]:
f['Acquisition']['Raw[0]']['RawData']

<HDF5 dataset "RawData": shape (47500, 12000), type "<i4">

In [6]:
# Let's just create a single file zarr

ds = xr.Dataset({
    'RawData':((['distance', 'time'], f['Acquisition']['Raw[0]']['RawData'][:])),
    'RawDataTime':('time', f['Acquisition']['Raw[0]']['RawDataTime'][:]),
    'GpBits':('time', f['Acquisition']['Raw[0]']['Custom']['GpBits']),
    'GpsStatus':('time', f['Acquisition']['Raw[0]']['Custom']['GpsStatus']),
    'PpsOffset':('time', f['Acquisition']['Raw[0]']['Custom']['PpsOffset']),
    'SampleCount':('time', f['Acquisition']['Raw[0]']['Custom']['SampleCount'])}
)

In [7]:
ds = ds.chunk({'time':3000, 'distance':3000})

In [8]:
fn = 'test.zarr'

ds.to_zarr(fn)

In [9]:
ds

<xarray.Dataset>
Dimensions:      (distance: 47500, time: 12000)
Dimensions without coordinates: distance, time
Data variables:
    RawData      (distance, time) int32 dask.array<chunksize=(3000, 3000), meta=np.ndarray>
    RawDataTime  (time) int64 dask.array<chunksize=(3000,), meta=np.ndarray>
    GpBits       (time) uint8 dask.array<chunksize=(3000,), meta=np.ndarray>
    GpsStatus    (time) uint8 dask.array<chunksize=(3000,), meta=np.ndarray>
    PpsOffset    (time) uint32 dask.array<chunksize=(3000,), meta=np.ndarray>
    SampleCount  (time) int64 dask.array<chunksize=(3000,), meta=np.ndarray>

### General Flow of script

For now, most of this process (except for actually writing the zarr uses just a single core)
I spent a little bit of time trying to open the file with fsspec (see section below) and not have to download the file to local disk but couldn't get it working.

- download single file to local disk
- read into RAM
- construct xr.Dataset
- append to zarr store
- delete single file from local disk

### Build Script

In [3]:
# setup access to Azure Storage
storage_options = {'account_name':'dasdata', 'account_key':os.environ['AZURE_KEY_dasdata']}
#ds = xr.open_zarr('abfs://zarr/SouthCable_Tx.zarr', storage_options=storage_options)

In [4]:
baseurl = url = 'http://piweb.ooirsn.uw.edu/das/data/Optasense/SouthCable/TransmitFiber/South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T16_09_15-0700/'

with open('file_list.txt') as f:
    fs = f.readlines()
    
files = []

for f in fs:
    files.append(f[:-1])

In [7]:
first_loop = True

for file in tqdm(files, position=0, leave=True):
    print('download file...', end="\r")
    os.system(f'aria2c -q {baseurl + file}')
    
    print('read data into memory and construct xr.Dataset ...', end="\r")
    hf = h5py.File(file)
    
    ds = xr.Dataset({
        'RawData':((['distance', 'time'], hf['Acquisition']['Raw[0]']['RawData'][:])),
        'RawDataTime':('time', hf['Acquisition']['Raw[0]']['RawDataTime'][:]),
        'GpBits':('time', hf['Acquisition']['Raw[0]']['Custom']['GpBits']),
        'GpsStatus':('time', hf['Acquisition']['Raw[0]']['Custom']['GpsStatus']),
        'PpsOffset':('time', hf['Acquisition']['Raw[0]']['Custom']['PpsOffset']),
        'SampleCount':('time', hf['Acquisition']['Raw[0]']['Custom']['SampleCount'])}
    )
    
    print('write to zarr...                                    ', end="\r")
    # create new zarr store if beginning of loop otherwize, append in time dimension
    if first_loop:
        first_loop = False
        ds.to_zarr('abfs://zarr/SouthCable_Tx.zarr', storage_options=storage_options, mode='w-')
    else:
        ds.to_zarr('abfs://zarr/SouthCable_Tx.zarr', storage_options=storage_options, append_dim='time')
    
    # delete downloaded file
    os.system(f'rm {file}')

  0%|          | 0/3981 [00:00<?, ?it/s]

  0%|          | 0/3981 [00:41<?, ?it/s]

ContainsGroupError: path '' contains a group

In [6]:
baseurl + file

'http://piweb.ooirsn.uw.edu/das/data/Optasense/SouthCable/TransmitFiber/South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T16_09_15-0700/South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T231114Z.h5'

- all that's left to do is create the storage account location for this...

### Playing with fsspec

- currently RSA server doesn't support range requests
    - see https://github.com/h5py/h5py/issues/2185
- currently in contact with RSA team to inquire if server can be changed to allow range requests

In [3]:
import fsspec

In [4]:
url = 'http://piweb.ooirsn.uw.edu/das/data/Optasense/SouthCable/TransmitFiber/South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T16_09_15-0700/South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T231514Z.h5'

In [12]:
of = fsspec.open(url, 'rb')

In [26]:
with of as of1:
    f = h5py.File(of1)
    print(f['Acquisition']['Raw[0]'].keys())

<KeysViewHDF5 ['Custom', 'RawData', 'RawDataTime']>
